In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F 
import re
import math
import os
import pandas as pd

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/opt/ml/input/data/'
file_name = 'test_feature_engineering.csv'

In [3]:
test_file_path = os.path.join(data_dir, file_name)
test = pd.read_csv(test_file_path)  # , nrows=100000)

In [4]:
md = TabNetMultiTaskClassifier(device_name='cpu')
md.load_model('./tabnetmodel.zip')

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [5]:
test.pop('Timestamp')

0         2020-01-09 10:56:31
1         2020-01-09 10:56:57
2         2020-01-09 10:58:31
3         2020-01-09 10:58:36
4         2020-01-09 10:58:43
                 ...         
260109    2020-10-14 23:07:23
260110    2020-10-14 23:07:41
260111    2020-10-14 23:08:02
260112    2020-10-14 23:09:31
260113    2020-10-14 23:10:03
Name: Timestamp, Length: 260114, dtype: object

In [6]:
test.pop('relative_answered_correctly')

0         0.340000
1         0.380000
2        -0.548000
3        -0.636000
4        -0.304000
            ...   
260109   -0.436667
260110    0.516667
260111    0.153333
260112    0.170000
260113   -1.605351
Name: relative_answered_correctly, Length: 260114, dtype: float64

In [7]:
test.pop('answerCode')

0         1.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
260109    0.0
260110    1.0
260111    1.0
260112    1.0
260113   -1.0
Name: answerCode, Length: 260114, dtype: float64

In [8]:
for col in tqdm(test.columns):
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col].values)

100%|██████████| 24/24 [00:00<00:00, 57.12it/s]


In [9]:
# md.device_name = 'cpu'

In [10]:
test_data = test[test['userID'] != test['userID'].shift(-1)].to_numpy()


In [11]:
loaded_preds = md.predict_proba(test_data)

In [12]:
preds = loaded_preds[0][:, 1]

In [13]:
output_dir = './output'
write_path = os.path.join(output_dir, "tabnetmulti.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)    
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : ./output/tabnetmulti.csv
